In [ ]:
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
# print('X_train: ' + str(train_X.shape))
# print('Y_train: ' + str(train_y.shape))
# print('X_test:  '  + str(test_X.shape))
# print('Y_test:  '  + str(test_y.shape))

In [ ]:
threshold = 50 ## for Binarize

def Binarize(X):
    X[X < threshold] = 0
    X[X >= threshold] = 1   
    return X

train_X = Binarize(train_X)
test_X = Binarize(test_X)

In [ ]:
def plot_digit(x, y, idx):
    img = x[idx]
    plt.imshow(img, cmap='Greys', interpolation='nearest')
    plt.title('Plot Number: %d' % y[idx])
    plt.show()


# plot_digit(train_X, train_y, 0)

In [ ]:
def centroid(arr):
    moments = {'M00': arr.sum(axis=1).sum(),
               'M01': (np.arange(start=1, stop=arr.shape[1] + 1, step=1) * arr.sum(axis=0).reshape(1, -1)).sum(),
               'M10': (np.arange(start=1, stop=arr.shape[0] + 1, step=1) * arr.sum(axis=1).reshape(1, -1)).sum(),
               'M11': 0}

    x = moments['M10'] / moments['M00'] if moments['M00'] != 0 else 0
    y = moments['M01'] / moments['M00'] if moments['M00'] != 0 else 0
    return [x, y]


In [ ]:
def imageWin(trainx, r, c):
    temp = []
    for i in range(0, int(trainx.shape[0]), r):
        for j in range(0, int(trainx.shape[1]), c):
            temp.append(centroid(trainx[i:i+r, j:j+c]))
    return np.array(temp)


def slicingArray(trainX, r, c):
    featureVector = []
    for trainx in trainX:
        featureVector.append(imageWin(trainx, r, c))
    return np.array(featureVector)

In [ ]:
def euclideanDistance(vec1, vec2):
	  return  np.sqrt(np.square(vec1 - vec2).sum())
	# dist = 0.0
	# for i in range(len(vec1) - 1):
	# 	dist += (vec1[i] - vec2[i])**2
	# return np.sqrt(dist)

class KNNClassifier:
    def __init__(self, k=3):
        self.x = None
        self.y = None
        self.k = k

    def fit(self, x: np.array, y: np.array):
        self.x = slicingArray(x, 4, 7)
        self.y = y

    def predict(self, testX):
        preds = []
        for i in range(len(testX)):
            distances = []
            for rowIdx in range(len(self.x)):
                dist = euclideanDistance(imageWin(testX[i], 4, 7), self.x[rowIdx])
                distances.append([dist, self.y[rowIdx]])
            distances.sort(key = lambda tup : tup[0])
            neighbors = []
            for j in range(self.k):
                neighbors.append(distances[j])
            out = [row[-1] for row in neighbors]
            preds.append(max(set(out), key=out.count))
        return preds

In [ ]:
## My KNN ##

knn = KNNClassifier(k=3)
train_X = slicingArray(train_X, 4, 7)

knn.fit(train_X[:15000],train_y[:15000])

processedTestX = slicingArray(test_X, 4, 7)

print("My KNN Accuracy:", accuracy_score(test_y,knn.predict(processedTestX)))

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
arr = slicingArray(train_X, 4, 4)
new_trainX = arr.reshape(*arr.shape[:-2], -1)

neigh.fit(new_trainX, train_y)

arr = slicingArray(test_X, 4, 4)
new_testX = arr.reshape(*arr.shape[:-2], -1)

print("SKlearn KNN Accuracy:", accuracy_score(test_y,neigh.predict(new_testX)))